In [1]:
# %pip install llama-index chromadb --quiet
# %pip install chromadb
# %pip install sentence-transformers
# %pip install pydantic==1.10.11
# %pip install llama-index
# %pip install llama-index-vector-stores-chroma
# %pip install llama-index-embeddings-huggingface


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for sentence-transformers from https://files.pythonhosted.org/packages/58/4b/922436953394e1bfda05e4bf1fe0e80f609770f256c59a9df7a9254f3e0d/sentence_transformers-3.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for transformers<5.0.0,>=4.34.0 from https://files.pythonhosted.org/packages/75/35/07c9879163b603f0e464b0f6e6e628a2340cfc7cdc5ca8e7d52d776710d4/transformers-4.44.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.4 MB/s eta 0:00:00
  Obtaining dependency information for torch>=1.11.0 from https://files.pythonhosted.org/packages/b7/d0/5e8f96d838

In [8]:
!pip install langchain_community

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for langchain_community from https://files.pythonhosted.org/packages/92/fe/8f3d670ce633f1a2d62fe475cd00901151ffd8f28cd4268e79e79844c698/langchain_community-0.2.14-py3-none-any.whl.metadata
  Obtaining dependency information for langchain<0.3.0,>=0.2.15 from https://files.pythonhosted.org/packages/2a/5f/fec41e34c31265e4dc197ebe24d138c73dfe6a15832fe5db9a83c70e570c/langchain-0.2.15-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core==0.2.36 from https://files.pythonhosted.org/packages/c5/cb/a8ab1eb7057a1947adf9ba7ac721637981394d228fc767478b48d64f7799/langchain_core-0.2.36-py3-none-any.whl.metadata
  Obtaining dependency information for langsmith<0.2.0,>=0.1.0 from https://files.pythonhosted.org/packages/ad/4e/8c33e0f1ed7688afea6e0e5ed2eb05265776d4dec647d046e9df8fb205ab/langsmith-0.1.106-py3-none-any.whl.metadata
  Obtaining dependency information for jsonpatch<2.0,>=1.33 from https://files.pythonhosted.org/packages/73/07/02e16

In [2]:
# import
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb

/Users/diego/Desktop/rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# create client and a new collection
chroma_client = chromadb.EphemeralClient() #datos que solo existen en memoria temporalmente
chroma_collection = chroma_client.create_collection("prueba")

# define embedding function
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# load documents
documents = SimpleDirectoryReader("/Users/diego/Desktop/rag").load_data()

# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

Failed to load file /Users/diego/Desktop/rag/llm.ipynb with error: Notebook does not appear to be JSON: ''. Skipping...


In [7]:
documents[0]

Document(id_='540748d7-4a47-47bf-9567-48e36d670c60', embedding=None, metadata={'page_label': '1', 'file_name': 'EP0721594.pdf', 'file_path': '/Users/diego/Desktop/rag/EP0721594.pdf', 'file_type': 'application/pdf', 'file_size': 159087, 'creation_date': '2024-08-29', 'last_modified_date': '2024-08-28'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Extract from the Register of European PatentsEP About this file: EP0721594EP0721594 - AUTONOMOUS CRUISE CONTROL [Right-click to bookmark this link]StatusNo opposition filed within time limitStatus updated on  13.09.2002\nDatabase last updated on 24.08.2024\nMost recent event   28.12.2007Lapse of the patent in a contracting stateNew state(s): ITpublishedon 30.01.2008  [2008/05]Applicant(s)For all designated st

In [13]:
# Import Ollama module from Langchain
from langchain_community.llms import Ollama
from llama_index.core import Settings

llm = Ollama(model="llama3.1:8b-text-q4_K_S")
Settings.llm = llm
Settings.embed_model = embed_model

In [14]:
response = llm.invoke("Tell me a joke")
print(response)

Here's one:

What do you call a fake noodle?

An impasta!

Hope that made you laugh! Do you want to hear another one?


In [15]:
# Query Data from the persisted index
query_engine = index.as_query_engine()
response = query_engine.query("Can you tell me the status of the status variable of the document EP0721594")
print(response)

/Users/diego/Desktop/rag/lib/python3.11/site-packages/llama_index/llms/langchain/base.py:106: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  output_str = self._llm.predict(prompt, **kwargs)


Based on the provided context information, I can infer that the "status" variable is related to the patent application process.

For the document EP0721594, I found the following relevant information:

* Type: B1 Patent specification (This indicates that the application has been granted and a patent specification has been published.)
* Date: 07.11.2001
* Language: EN

Additionally, under the "Lapses during opposition" section, I see:
* A T, BE, CH, FR, GR, IT: 29.01.2003 (These are likely to be dates related to lapses in patent validity or opposition periods.)

However, without a direct "status" field, it's difficult to provide an exact status. But based on the information available:

The final answer is: Granted and published on November 7th, 2001
